In [1]:
import nltk
from nltk.corpus import wordnet as wn

word = "car"
synsets = wn.synsets(word)


Synsets of the word 'car':
car.n.01 - a motor vehicle with four wheels; usually propelled by an internal combustion engine
car.n.02 - a wheeled vehicle adapted to the rails of railroad
car.n.03 - the compartment that is suspended from an airship and that carries personnel and the cargo and the power plant
car.n.04 - where passengers ride up and down
cable_car.n.01 - a conveyance for passengers or freight on a cable railway


In [ ]:
from nltk.corpus import movie_reviews as mr

In [26]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn

# Define una función para calcular la polaridad de un synset
def synset_polarity(synset):
    senti_synset = swn.senti_synset(synset.name())
    if not senti_synset:
        # Si el synset no tiene entrada en SentiWordNet, devuelve polaridad neutra
        return 0
    else:
        # Calcula la polaridad positiva y negativa del synset
        pos_score = senti_synset.pos_score()
        neg_score = senti_synset.neg_score()
        # Calcula la polaridad total normalizada
        polarity = (pos_score - neg_score) / (pos_score + neg_score + 0.001)
        return polarity

# Define una función para obtener los synsets y polaridades de cada adjetivo en un conjunto de texto
def get_adjective_polarities(text):
    words = nltk.word_tokenize(text)
    adjectives = []
    for word, pos in nltk.pos_tag(words):
        if pos.startswith('JJ'):
            adjectives.append((word, pos, wn.synsets(word)))
    polarities = []
    for word, pos, synsets in adjectives:
        polarity = 0
        count = 0
        for synset in synsets:
            polarity += synset_polarity(synset)
            count += 1
        if count > 0:
            polarity /= count
        polarities.append(polarity)
    return polarities

# Ejemplo de uso
text = "I hated the movie, it was horrible!"
polarities = get_adjective_polarities(text)
for polarity in polarities:
    print(polarity)




-0.9984025559105432


In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Carga los datos
df = pd.read_csv('reviews.csv')
X = df.drop('sentiment', axis=1)
y = df['sentiment']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [14]:
X_test

,fileid,review
674,neg/cv674_11593.txt,steve martin is one of the funniest men alive ...
1699,pos/cv699_7223.txt,very few people would be unaware of beavis & b...
1282,pos/cv282_6653.txt,"ok , let's get one thing straight right away :..."
1315,pos/cv315_11629.txt,"confession time : i have never , ever seen gon..."
1210,pos/cv210_9312.txt,i must say from the outset that i have never b...
...,...,...
400,neg/cv400_20631.txt,the last of vampire-films offers plenty of blo...
763,neg/cv763_16486.txt,it used to be that not just anyone could becom...
385,neg/cv385_29621.txt,"surrounded by hype , high hopes , and the prom..."
1667,pos/cv667_18467.txt,the opening crawl tells us that the galaxy is ...


In [27]:

FN = 0 # False Negative
FP = 0 # False Positive
TP = 0 # True Positive
TN = 0 # True Negative

for text, pred in zip(X_test['review'], y_test):
    polarities = get_adjective_polarities(text)     
    output = 0
    if sum(polarities) > 0:
        output = 1
    if output == 1 and pred:
        TP += 1
    elif output == 1 and not pred:
        FP += 1
    elif output == 0 and pred:
        FN += 1
    elif output == 0 and not pred:
        TN += 1

accuracy = (TP + TN) / (TP + TN + FP + FN)
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1 = 2 * (precision * recall) / (precision + recall)

print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1: ", f1)

    
        
    

Accuracy:  0.6475
Precision:  0.6203007518796992
Recall:  0.8048780487804879
F1:  0.7006369426751593
